Importe de librerias y carga de datos

In [1]:
import pandas as pd
import numpy as np
import ast
import json
import re
import datetime as dt


eda = pd.read_csv(r"C:\Users\migue\OneDrive\Escritorio\movies_dataset.csv")

C:\Users\migue\AppData\Local\Temp\ipykernel_14412\3655119525.py:9: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  eda = pd.read_csv(r"C:\Users\migue\OneDrive\Escritorio\movies_dataset.csv")


In [2]:
# Eliminación primeras columnas que no serán utilizadas
columnas_eliminar = ['video', "imdb_id", "adult", "original_title", "poster_path", "homepage"]
eda.drop(columns=columnas_eliminar, inplace=True)

# eliminacion de duplicados
eda = eda.drop_duplicates()

# Se eliminan 87 entradas que no tienen release_date
eda.dropna(subset=['release_date'], inplace=True)


#Se deciden eliminar las 3 entradas de revenue 0 por data insuficiente
eda.dropna(subset=['revenue'], inplace=True)


# Creamos columa return (revenue/budget) que dará 0 en caso de no poder realizar la división
#primero creamos la función que calculará el return
def cal_return(revenue, budget):
    if float(budget) == 0:
        return 0
    else:
        return float(revenue)/float(budget)
    
#creamos nueva columa return
eda['return']=eda.apply(lambda row: cal_return(row['revenue'], row['budget']), axis=1)


# transformamos las fechas de release_date a datetime
eda['release_date']=pd.to_datetime(eda['release_date'])
#eda['release_date']=eda['release_date'].dt.strftime('%Y-%m-%d')  #Esto es para volverlo str con formato AAAA-mm-dd

# se crea columna 'release_year' donde se alberga solo el año de estreno
eda['release_year']=eda['release_date'].dt.year


# funcion para obtener el día de la semana dada una fecha AAAA-mm-dd
def obtener_dia_semana(fecha):
    # Convertir la cadena de fecha en un objeto datetime
    fecha_obj = fecha
    # Obtener el día de la semana como una cadena
    dia_semana = fecha_obj.strftime('%A')
    return dia_semana

#se crea una columna con los días de estreno
eda['release_day']=eda['release_date'].apply(obtener_dia_semana)



In [3]:
# DESANIDADO DE Genre

# Se crea una función que reemplaza las comillas ' por " para manejar json
def reemplazar_comillas(cadena):
  return re.sub(r"'", '"', cadena)

# Se cambian comillas de columna genres
eda['genres'] = eda['genres'].apply(reemplazar_comillas)

# Se transforma los datos de str a listas
eda['genres'] = eda['genres'].apply(lambda x: json.loads(x))

# Se desanidan id y los nombres de la columna genres
eda['genres_ids'] = eda['genres'].apply(lambda x: [d['id'] for d in x])
eda['genres_names'] = eda['genres'].apply(lambda x: [d['name'] for d in x])

In [4]:
# DESANIDADO DE production_companies, production_countries y spoken_language

# funcion para formatear las conlumnas con listas de diccionarios
def convert_to_list_of_dicts(value):
    try:
        value = value.strip()
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        return [] 

# Se transforma la columna production_companies y se desanida en id y names
eda['production_companies'] = eda['production_companies'].apply(convert_to_list_of_dicts)
eda['production_companies_id'] = eda['production_companies'].apply(lambda x: [d['id'] for d in x])
eda['production_companies_names'] = eda['production_companies'].apply(lambda x: [d['name'] for d in x])


# de la columna production_countries se desanida el codigo iso
eda['production_countries'] = eda['production_countries'].apply(convert_to_list_of_dicts)
eda['production_countries_iso'] = eda['production_countries'].apply(lambda x: [d['iso_3166_1'] for d in x])
eda['production_countries_names'] = eda['production_countries'].apply(lambda x: [d['name'] for d in x])

# se transforma la columna spoken_languages a lista de dicc y se desanidan
eda['spoken_languages'] = eda['spoken_languages'].apply(convert_to_list_of_dicts)
eda['spoken_languages_iso'] = eda['spoken_languages'].apply(lambda x: [d['iso_639_1'] for d in x])
eda['spoken_languages_name'] = eda['spoken_languages'].apply(lambda x: [d['name'] for d in x])

In [5]:
# DESANIDADO DE belongs_to_collection

# Se reemplazan los Nan de belongs_to_collection por dicc vacios
#eda['belongs_to_collection']=eda['belongs_to_collection'].fillna('{}') #opcion 1 funciona pero son str y falla luego
eda['belongs_to_collection']=eda['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else {})

# se crean funciones para extraer id y name de los diccionarios
def extract_id(row):
    try:
        return row['id']
    except:
        return None

def extract_name(row):
    try:
        return row['name']
    except:
        return None
    
# se crean nuevas columnas con los id y name extraidos
eda['belongs_to_collection_id']=eda['belongs_to_collection'].apply(extract_id)
eda['belongs_to_collection_name']=eda['belongs_to_collection'].apply(extract_name)

In [6]:
# Se eliminan columnas originales desanidadas

# Eliminación columnas desanidadas
columnas_eliminar = ['belongs_to_collection', "genres", "production_countries", "production_companies", "spoken_languages"]
eda.drop(columns=columnas_eliminar, inplace=True)

Segundo archivo: credits.csv

In [7]:
# Se carga el segundo archivo
eda2 = pd.read_csv(r"C:\Users\migue\OneDrive\Escritorio\credits.csv")

In [8]:
# se formatea columna cast y se desanida en los actores/actrices
eda2['cast'] = eda2['cast'].apply(convert_to_list_of_dicts)
eda2['cast_names'] = eda2['cast'].apply(lambda x: [d['name'] for d in x])


In [9]:
# se crea una función para extraer director de la columna crew
def extraer_nombre_director(lista_diccionarios):
  for diccionario in lista_diccionarios:
    if diccionario['job'] == 'Director':
      return diccionario['name']
  return None

# Se transforma crew a lista de dicc y se le aplica el extractor de director para crear nueva columna
eda2['crew'] = eda2['crew'].apply(convert_to_list_of_dicts)
eda2['director_name']=eda2['crew'].apply(extraer_nombre_director)

In [10]:
# se eliminan columnas cast y crew originales
columnas_eliminar = ['crew', 'cast']
eda2.drop(columns=columnas_eliminar, inplace=True)


In [11]:
# transformamos el dtype de columna 'id' a int64 para poder hacer merge
eda['id'] = eda['id'].astype('int64')

# Merge de eda y eda2 en un solo df
df = eda.merge(eda2, on='id', how='inner')

In [12]:
# Finalmente se guarda el dataset desanidado, limpio y filtrado

#df.to_csv('limpio1.csv', index=False)